<center>
    <img src="https://i.ibb.co/wQg0qzq/Banner.png" width="900" alt="KCode Banner"  />
</center>


# Algoritmo de regressão linear simples

Vamos utilizar um conjunto de dados referente à emissão de CO2 por automóveis
# Objetivo:
* Utilizar a bilioteca scikit-learn para implantar um modelo de regressão linear simples;
* Criar um modelo, treina-lo, testa-lo e utilizar.

### Importando bibliotecas necessárias

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
import requests as rq
%matplotlib inline

### Criando o DataFrame

In [8]:
df = pd.read_csv("FuelConsumptionCo2.csv")

# take a look at the dataset
df.head()


,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2014,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,2014,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,2014,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,2014,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,2014,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


## Entendendo os dados

### `FuelConsumptionCo2.csv`:


Fizemos o download de um conjunto de dados, **`FuelConsuptionCo2.csv`**, de consumo de combustível, que contém consumo de combustível de modelos específicos e taxas estimadas de emissão de dioxido de carbono, para veículos leves novos à venda varejo no Canadá. [Dataset source](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01)


*   **MODELYEAR** e.g. 2014 - **Ano do Modelo**
*   **MAKE** e.g. Acura - **Fabricante**
*   **MODEL** e.g. ILX - **Modelo**
*   **VEHICLE CLASS** e.g. SUV - **Classe do veículo**
*   **ENGINE SIZE** e.g. 4.7 - **Tamanho do motor**
*   **CYLINDERS** e.g 6 - **Número de cilindros**
*   **TRANSMISSION** e.g. A6 - Transmissão
*   **FUEL CONSUMPTION in CITY(L/100 km)** e.g. 9.9 - **Consumo de combústivel L/100km na cidade**
*   **FUEL CONSUMPTION in HWY (L/100 km)** e.g. 8.9 - **Consumo de combústivel L/100km na BR**
*   **FUEL CONSUMPTION COMB (L/100 km)** e.g. 9.2 - **Consumo de combústivel L/100km Combinado**
*   **CO2 EMISSIONS (g/km)** e.g. 182   --> low -->  - **Emissão de Co2** 

### Para melhor compreensão, vamos agora traduzir os títulos das colunas (headers), para o português